In [1]:
from flask import Flask, request, jsonify

import time
import os
import azure.cognitiveservices.speech as speechsdk
import json 
from datetime import datetime 
import subprocess
from pydub.utils import mediainfo
import json
import cv2
import os, shutil
import boto3
from io import StringIO
import sys
import time
from moviepy.editor import VideoFileClip
import threading
import re
import torch
# from memory_profiler import profile
# import gc
# import matplotlib.pyplot as plt
from PIL import Image
import cv2
import torch.hub
import time
import cv2
import torch
from google.oauth2 import service_account
import json
from google.cloud import vision

# from flask import Flask, request, jsonify
# import os
# import shutil
# from moviepy.editor import VideoFileClip

In [2]:
def convert_video_to_wav(video_path):
    video_data = mediainfo(video_path)
    channels = video_data["channels"]
    bit_rate = video_data["bit_rate"]
    sample_rate = video_data["sample_rate"]
    wav_output = video_path.replace(".mp4",".wav").replace("video","wav_file")

    ffmpeg_cmd = ['ffmpeg', '-i', video_path,  '-b:a', bit_rate, '-ar', sample_rate , '-ac', channels ,'-vn', wav_output]
    
    try:
        subprocess.check_output(ffmpeg_cmd, stderr=subprocess.STDOUT)
        print("Video converted to WAV successfully!")
        return wav_output
    except subprocess.CalledProcessError as e:
        print("Error converting video to WAV:", e.output)
        return None


In [3]:
# def transcribe(key,region,lang,path_in,path_out="out.txt",newLine=False):auto_detect_source_language_config
def transcribe(key,region,auto_detect_source_language_config,path_in,path_out="out.txt",newLine=False):
    # lang = auto_detect_source_language_config
    speech_config = speechsdk.SpeechConfig(subscription=key, region=region)
    # speech_config.speech_recognition_language=lang
    speech_config.output_format = speechsdk.OutputFormat(1)
    speech_config.request_word_level_timestamps()
    audio_config = speechsdk.AudioConfig(filename=path_in)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config,auto_detect_source_language_config=auto_detect_source_language_config, audio_config=audio_config)
    done = False
    tep_text_json = []
    textOut = ""
    def stop_cb(evt):
        print(1)
        print(evt)
        speech_recognizer.stop_continuous_recognition()
        nonlocal done
        done = True

    str_newLine = ""
    if newLine:
        str_newLine = " \n"

    def outPrint(evt):
        nonlocal textOut
        nonlocal tep_text_json
        tmp_text = evt.result.text
        tmp_timestamp = evt.result.offset
        tmp_duration_ts = evt.result.duration
        textOut += f"[{tmp_timestamp}] [{tmp_duration_ts}] {tmp_text}" + str_newLine
        
        out = eval(evt.result.json)
        tep_text_json.append(out)
        # print(2)
        # print(tmp_text)
        # print(3)
        # print(tmp_timestamp)
        # print(4)
        # print(tmp_duration_ts)
        # print(5)
        # print('-' * 100)
        # print(evt)
        # print('-' * 100)
        # print(evt.result)
        # print('-' * 100)
        # print("haa yahi h be")
        # print(evt.result.json)
        # out = eval(evt.result.json)
        # print(type(out))
        # print(out)
        # print('-' * 100)
        # print(evt.result.properties)
        # print('-' * 100)
        # print(dir(evt))
        # print('-' * 100)
        # print(dir(evt.result))
    
    speech_recognizer.recognized.connect(outPrint)
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.start_continuous_recognition()

    while not done:
        time.sleep(.5)
    # with open(path_out, 'w', encoding='utf-16') as f:
    #     f.write(textOut)
    # tep_text_json = str(tep_text_json)
    # path_out="./"
    print("tep_text_json:",tep_text_json)
    json_data = json.dumps(tep_text_json)
    print("json_data:",json_data)
    with open(path_out, 'w', encoding='utf-8') as f:
        f.write(json_data)

In [4]:
def milliseconds_to_srt_timestamp(milliseconds):
    total_seconds = milliseconds // 1000
    # milliseconds %= 1000
    # minutes = seconds // 60
    # seconds %= 60
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    milliseconds = milliseconds % 1000
    # minutes %= 60
    return '{:02d}:{:02d}:{:02d},{:03d}'.format(hours, minutes, seconds, milliseconds)


# json_file = "D:/Python_scripts/speech_to_test_azure/New folder/txt_&_json_file/txt_&_json_file/Piyush_2.json"  # Replace with the path to your JSON file
def convert_json_into_words_by_words_list(json_file):
    print("adsefesdwefewdwefwdwedbadja bseujwsduwbdakuscbasukdasbdaeudaeudavsxuasvbaoudygwuoqevwqb8dyasdwvwe")
    with open(json_file, "r",encoding='utf-8') as file:
        json_output = file.read()

    data = json.loads(json_output)
    # print(len(data))
    print(data)
    srt_content = ""
    list_of_words = []
    for j in range(len(data)):
        # print(j)
    # Extract word-level details
        try:
            words = data[j]["NBest"][0]["Words"]

        # Generate SRT content
            for i, word in enumerate(words):
                list_with_time =[]
                start_timestamp = milliseconds_to_srt_timestamp(word["Offset"] // 10000)
                end_timestamp = milliseconds_to_srt_timestamp((word["Offset"]  + word["Duration"]) // 10000)
                subtitle_text = word["Word"]
                srt_entry = f"{i+1}\n{start_timestamp} --> {end_timestamp}\n{subtitle_text}\n\n"
                srt_content += srt_entry
                list_with_time.append(start_timestamp)
                list_with_time.append(end_timestamp)
                list_with_time.append(subtitle_text)
                list_of_words.append(list_with_time)
        except:
            pass
    # json_data = json.dumps(list_of_words)
    print("json_data "*80)
    # json_data = json.loads(json_data)
    with open("./list_of_words.json","w",encoding="utf-8") as f:
        json.dump(list_of_words,f,ensure_ascii=False)
    return list_of_words

# json_file = "D:/Python_scripts/speech_to_test_azure/srt_fol/pm_modi.json"

# list_of_words = convert_json_into_words_by_words_list(json_file)

In [5]:

def convert_time_stamp_to_seconds(time_stamp):
    time_format = "%H:%M:%S,%f"

    # Convert the time stamp to a datetime object
    time_obj = datetime.strptime(time_stamp, time_format)

    # Calculate the total number of seconds
    seconds = time_obj.hour * 3600 + time_obj.minute * 60 + time_obj.second + time_obj.microsecond / 1000000

    return seconds

In [6]:
def generate_srt_file(list_of_words,filename):

    list_of_new_srt = []
    str_of_words = ""
    total_words_covered = 0
    index = 1
    srt_sentence = ""

    list_for_sentences_for_the_final_srt =[]
    while total_words_covered != len(list_of_words):
        transcription_list = []
        start_point = convert_time_stamp_to_seconds(list_of_words[total_words_covered][0])
        start_for_srt = list_of_words[total_words_covered][0]

        transcription_list.append(list_of_words[total_words_covered][2])
        total_words_covered += 1

        
        list_for_srt_by_sentence = []
        
        dict = {}

        # DECIDE AN END FOR START WORD

        for j in range(total_words_covered, (len(list_of_words) - 1)):

            end_point =convert_time_stamp_to_seconds(list_of_words[j][1])
            end_for_srt = list_of_words[j][1]
            end_for_prev_srt = list_of_words[j - 1][1]
        
            time_limit = round(((end_point) - (start_point)),3)
            
            
            # bin_size
            if time_limit <= (2.50):
                transcription_list.append(list_of_words[j][2])
                total_words_covered += 1
                end_for_srt = list_of_words[j][1]

            else:
                list_for_srt_by_sentence.append(transcription_list)
                dict["transcript"] = transcription_list
                dict["start_point"] = start_for_srt
                dict["end_point"]  = end_for_prev_srt
                break
    

        list_of_new_srt.append(dict)
        str_of_words = ""
    for transcript_dict in list_of_new_srt:
        dict ={}
        if "transcript" in transcript_dict.keys():
            subtitle_text = (' '.join(transcript_dict["transcript"]))
            start_time = transcript_dict["start_point"]
            end_time = transcript_dict["end_point"] 
            dict["transcript"] = subtitle_text
            dict["start_point"] = start_time
            dict["end_point"] = end_time
            list_for_sentences_for_the_final_srt.append(dict)

    for dict_sentences in list_for_sentences_for_the_final_srt:
        # print(dict_sentences)
        subtitle_text = dict_sentences["transcript"]
        start_timestamp = dict_sentences['start_point']
        end_timestamp = dict_sentences['end_point']
        srt_entry = f"{index}\n{start_timestamp} --> {end_timestamp}\n{subtitle_text}\n\n"
        index +=1
        # print(srt_entry)
        # break
        srt_sentence += srt_entry

        # srt_content += ""+ srt_entry

    filename = filename

    with open(filename, 'w',encoding="utf-8") as file:
        file.write(srt_sentence)
    return srt_sentence

    
    

In [8]:

class VideoDetect:
    jobId = ''
    rek = boto3.client('rekognition', region_name='ap-south-1')
    sqs = boto3.client('sqs', region_name='ap-south-1')
    sns = boto3.client('sns', region_name='ap-south-1')

    roleArn = ''
    bucket = ''
    video = ''
    startJobId = ''

    sqsQueueUrl = ''
    snsTopicArn = ''
    processType = ''

    def __init__(self, role, bucket, video, segment):
        self.roleArn = role
        self.bucket = bucket
        self.video = video
        self.segment = segment


    def GetSQSMessageSuccess(self):

        jobFound = False
        succeeded = False

        dotLine=0
        while jobFound == False:
            sqsResponse = self.sqs.receive_message(QueueUrl=self.sqsQueueUrl, MessageAttributeNames=['ALL'],
                                                   MaxNumberOfMessages=10)

            if sqsResponse:

                if 'Messages' not in sqsResponse:
                    if dotLine<40:
                        print('.', end='')
                        dotLine=dotLine+1
                    else:
                        print()
                        dotLine=0
                    sys.stdout.flush()
                    time.sleep(5)
                    continue

                for message in sqsResponse['Messages']:
                    notification = json.loads(message['Body'])
                    rekMessage = json.loads(notification['Message'])
                    print(rekMessage['JobId'])
                    print(rekMessage['Status'])
                    if rekMessage['JobId'] == self.startJobId:
                        # print('Matching Job Found:' + rekMessage['JobId'])
                        jobFound = True
                        if (rekMessage['Status']=='SUCCEEDED'):
                            succeeded=True

                        self.sqs.delete_message(QueueUrl=self.sqsQueueUrl,
                                                ReceiptHandle=message['ReceiptHandle'])
                    else:
                        pass
                        # print("Job didn't match:" +
                        #       str(rekMessage['JobId']) + ' : ' + self.startJobId)
                    # Delete the unknown message. Consider sending to dead letter queue
                    self.sqs.delete_message(QueueUrl=self.sqsQueueUrl,
                                            ReceiptHandle=message['ReceiptHandle'])


        return succeeded

    # def StartCelebrityRecognition(self):
    def StartContentModeration(self):
        # response=self.rek.start_label_detection(Video={'S3Object': {'Bucket': self.bucket, 'Name': self.video}},
        #                                         NotificationChannel={'RoleArn': self.roleArn, 'SNSTopicArn': self.snsTopicArn})
        # response=self.rek.start_celebrity_recognition(Video={'S3Object': {'Bucket': self.bucket, 'Name': self.video}},
        print(self.bucket)
        print(self.video)
        response=self.rek.start_content_moderation(Video={'S3Object': {'Bucket': self.bucket, 'Name': self.video}},
                                                   NotificationChannel={'RoleArn': self.roleArn, 'SNSTopicArn': self.snsTopicArn})

        self.startJobId=response['JobId']
        # print('Start Job Id: ' + self.startJobId)
    def StartCelebrityRecognition(self):
        # response=self.rek.start_label_detection(Video={'S3Object': {'Bucket': self.bucket, 'Name': self.video}},
        #                                         NotificationChannel={'RoleArn': self.roleArn, 'SNSTopicArn': self.snsTopicArn})
        # response=self.rek.start_celebrity_recognition(Video={'S3Object': {'Bucket': self.bucket, 'Name': self.video}},
        print(self.bucket)
        print(self.video)
        response=self.rek.start_celebrity_recognition(Video={'S3Object': {'Bucket': self.bucket, 'Name': self.video}},
                                                   NotificationChannel={'RoleArn': self.roleArn, 'SNSTopicArn': self.snsTopicArn})

        self.startJobId=response['JobId']
        # print('Start Job Id: ' + self.startJobId)


    # def GetCelebrityRecognitionResults(self):
    def GetContentModerationResults(self):
        maxResults = 100
        paginationToken = ''
        # finished = False
        #
        # while finished == False:
        response = self.rek.get_content_moderation(JobId=self.startJobId,
                                                   MaxResults=maxResults,
                                                   NextToken=paginationToken,
                                                   SortBy='TIMESTAMP')

        print('Codec: ' + response['VideoMetadata']['Codec'])
        # print('Duration: ' + str(response['VideoMetadata']['DurationMillis']))
        print('Format: ' + response['VideoMetadata']['Format'])
        print('Frame rate: ' + str(response['VideoMetadata']['FrameRate']))
        print()

        result_list = []

        for moderation_label_info in response['ModerationLabels']:
            moderation_label_dict = {}
            segment_offset_seconds = (self.segment - 1) * 60000
            timestamp = moderation_label_info["Timestamp"]
            timestamp += segment_offset_seconds
            label_name = moderation_label_info["ModerationLabel"]["Name"]
            confidence = moderation_label_info["ModerationLabel"]["Confidence"]

            milliseconds = timestamp
            seconds,milliseconds = divmod(milliseconds,1000)
            minutes,seconds = divmod(seconds,60)
            hours, minutes = divmod(minutes,60)
            timestamp_proper_format = f"{hours:02d}:{minutes:02d}:{seconds:02d}"
            moderation_label_dict['timestamp_in_Hours_min_sec'] = timestamp_proper_format
            moderation_label_dict['timestamp'] = timestamp
            moderation_label_dict['Moderated Label Name'] = label_name
            moderation_label_dict['Confidence'] = confidence

            result_list.append(moderation_label_dict)

            print('Timestamp: ', timestamp, '\tModerated Label: ', label_name, "\t Confidence: ", confidence)
        return result_list

    def GetCelebrityRekognitionResults(self):
        maxResults = 100
        paginationToken = ''
        # finished = False
        #
        # while finished == False:
        response = self.rek.get_celebrity_recognition(JobId=self.startJobId,
                                                   MaxResults=maxResults,
                                                   NextToken=paginationToken,
                                                   SortBy='TIMESTAMP')

        print('Codec: ' + response['VideoMetadata']['Codec'])
        # print('Duration: ' + str(response['VideoMetadata']['DurationMillis']))
        print('Format: ' + response['VideoMetadata']['Format'])
        print('Frame rate: ' + str(response['VideoMetadata']['FrameRate']))
    
        # print()

        result_list = []

        for celeb_name in response['Celebrities']:
            celeb_dict = {}
            segment_offset_seconds = (self.segment - 1) * 60000
            timestamp = celeb_name["Timestamp"]
            timestamp += segment_offset_seconds
            name = celeb_name["Celebrity"]["Name"]
            confidence = celeb_name["Celebrity"]["Confidence"]

            milliseconds = timestamp
            seconds,milliseconds = divmod(milliseconds,1000)
            minutes,seconds = divmod(seconds,60)
            hours, minutes = divmod(minutes,60)
            timestamp_proper_format = f"{hours:02d}:{minutes:02d}:{seconds:02d}"

            celeb_dict['timestamp_in_proper_format'] = timestamp_proper_format
            celeb_dict['timestamp'] = timestamp
            celeb_dict['Celeb Name'] = name
            celeb_dict['Confidence'] = confidence

            result_list.append(celeb_dict)

            # print('Timestamp: ', timestamp, '\tModerated Label: ', label_name, "\t Confidence: ", confidence)
        return result_list



    def CreateTopicandQueue(self):

        millis = str(int(round(time.time() * 1000)))

        #Create SNS topic

        snsTopicName="AmazonRekognitionExample" + millis

        topicResponse=self.sns.create_topic(Name=snsTopicName)
        self.snsTopicArn = topicResponse['TopicArn']

        #create SQS queue
        sqsQueueName="AmazonRekognitionQueue" + millis
        self.sqs.create_queue(QueueName=sqsQueueName)
        self.sqsQueueUrl = self.sqs.get_queue_url(QueueName=sqsQueueName)['QueueUrl']

        attribs = self.sqs.get_queue_attributes(QueueUrl=self.sqsQueueUrl,
                                                AttributeNames=['QueueArn'])['Attributes']

        sqsQueueArn = attribs['QueueArn']

        # Subscribe SQS queue to SNS topic
        self.sns.subscribe(
            TopicArn=self.snsTopicArn,
            Protocol='sqs',
            Endpoint=sqsQueueArn)

        #Authorize SNS to write SQS queue
        policy = """{{
  "Version":"2012-10-17",
  "Statement":[
    {{
      "Sid":"MyPolicy",
      "Effect":"Allow",
      "Principal" : {{"AWS" : "*"}},
      "Action":"SQS:SendMessage",
      "Resource": "{}",
      "Condition":{{
        "ArnEquals":{{
          "aws:SourceArn": "{}"
        }}
      }}
    }}
  ]
}}""".format(sqsQueueArn, self.snsTopicArn)

        response = self.sqs.set_queue_attributes(
            QueueUrl = self.sqsQueueUrl,
            Attributes = {
                'Policy' : policy
            })

    def DeleteTopicandQueue(self):
        self.sqs.delete_queue(QueueUrl=self.sqsQueueUrl)
        self.sns.delete_topic(TopicArn=self.snsTopicArn)


def delete_temp_file(file_path):
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))


In [9]:
import re
def extract_segment_number(filename):
    pattern = r'_segment_(\d+)\.mp4'
    match = re.search(pattern, filename)
    if match:
        segment_number = int(match.group(1))
        return segment_number
    else:
        return None

In [10]:
def split_video_into_segments(input_file, segment_duration=60):
    try:
        # Construct the FFmpeg command
        ffmpeg_cmd = [
            'ffmpeg',
            '-i', input_file,
            '-c:v', 'libx264',
            '-segment_time', str(segment_duration),
            '-reset_timestamps', '1',
            '-f', 'segment',
            f'{input_file}_segment_%03d.mp4'
        ]
        
        # Run the FFmpeg command
        subprocess.run(ffmpeg_cmd, check=True)

        print("Video splitting completed successfully!")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {str(e)}")


In [11]:
# def split_video_into_segments(input_file, segment_duration=60):
#     try:
#         clip = VideoFileClip(input_file)

#         # Calculate the total duration of the input video in seconds
#         total_duration = clip.duration

#         # Calculate the number of segments required
#         num_segments = int(total_duration / segment_duration)

#         # Extract the file name and extension from the input_file
#         file_name, file_extension = input_file.split('.')

#         # Split the video into segments
#         for i in range(num_segments):
#             start_time = i * segment_duration
#             end_time = min((i + 1) * segment_duration, total_duration)
#             segment_clip = clip.subclip(start_time, end_time)
#             output_file = f"{file_name}_segment_{i + 1}.{file_extension}"
#             segment_clip.write_videofile(output_file, codec="libx264")

#         # For the last segment (if it's shorter than segment_duration)
#         if total_duration % segment_duration > 0:
#             start_time = num_segments * segment_duration
#             end_time = total_duration
#             last_segment_clip = clip.subclip(start_time, end_time)
#             output_file = f"{file_name}_segment_{num_segments + 1}.{file_extension}"
#             last_segment_clip.write_videofile(output_file, codec="libx264")

#         clip.close()
#         print("Video splitting completed successfully!")
#     except Exception as e:
#         print(f"An error occurred: {str(e)}")

In [12]:
def split_video_and_upload_s3(video_path):
    cwd = os.getcwd()
    listdir_earlier = os.listdir(cwd)
    print(listdir_earlier)
    # ffmpeg_cmd = ['ffmpeg', '-i', video_path,  '-c', 'copy', '-map 0', '-segment_time' , '00:01:00', "-f segment -reset_timestamps 1 output%03d.mp4"]
    # ffmpeg_split_cmd = """ffmpeg -i '""" + video_path + """' -c copy -map 0 -segment_time 00:01:00 -f segment -reset_timestamps 1 output%03d.mp4"""
    ffmpeg_split_cmd = """ffmpeg -i 'video/THOMAS_SHELBY_SMOKING.mp4' -c copy -map 0 -segment_time 00:01:00 -f segment -reset_timestamps 1 output%03d.mp4 > /dump 2>&1"""
    # subprocess.check_output(ffmpeg_cmd, stderr=subprocess.STDOUT)
    # subprocess.run(ffmpeg_split_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    try:
        print(ffmpeg_split_cmd)
        subprocess.check_output(ffmpeg_split_cmd, shell=True, stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError as e:
        raise RuntimeError("command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output))

    listdir_now = os.listdir(cwd)
    print(listdir_now)

    # cmd = 'aws s3 cp D:/Python_scripts/AWS_Scripts/my_function/my_deployment_package.zip s3://alt-cap-logsbucket-12leb47kv5fyr/'

In [13]:
def upload_to_s3(video_path):
    cmd = 'aws s3 cp "' + video_path + '" s3://<<aws_bucket_name>>'
    # subprocess.run(cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    try:
        subprocess.check_output(cmd, shell=True, stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError as e:
        raise RuntimeError("command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output))
    finally:
            delete_temp_file(video_path)
    

In [14]:
def get_videos_with_nomenclature():
    eligible_videos = []
    for filename in os.listdir('D:/Demo_Resources/Moderation_AI/video'):
        if filename.endswith('.mp4') and '_segment_' in filename:
            filename = 'D:/Demo_Resources/Moderation_AI/video/' + filename
            eligible_videos.append(filename)
            upload_to_s3(filename)  # Upload the video to S3 (you need to implement this function)
    return eligible_videos

In [15]:
# def Celebrity_reko(list_video_segments):
#     roleArn = 'arn:aws:iam::<aws_account_id>:role/AmazonRekognitionVideoServiceRole'
#     bucket = '<aws_bucket_name>'
#     # Initialize the analyzer
#     analyzer = VideoDetect(roleArn, bucket, "", 0)
#     # Create SNS topic and SQS queue once
#     analyzer.CreateTopicandQueue()
#     final_output_of_cele_rec = []
#     for video_segment in list_video_segments:
#         video = video_segment.split('/')[-1]
#         segment = extract_segment_number(video)
#         # Update the video and segment for the analyzer
#         analyzer.video = video
#         analyzer.segment = segment
#         print('Content Moderation Summary:')
#         # The following line was commented out in your code, but I assume you want to use it
#         # analyzer.StartCelebrityRecognition()
#         if analyzer.GetSQSMessageSuccess():
#             output_of_Celebrity_rec = analyzer.GetCelebrityRekognitionResults() 
#             final_output_of_cele_rec.append(output_of_Celebrity_rec)
#     # Delete SNS topic and SQS queue after processing all segments
#     analyzer.DeleteTopicandQueue()
#     return final_output_of_cele_rec


# def content_mode(list_video_segments):
    
#         roleArn = 'arn:aws:iam::<aws_account_id>:role/AmazonRekognitionVideoServiceRole'
#         bucket = '<aws_bucket_name>'

#         # split_video_into_segments(path_of_video)
#         # list_video_segments = get_videos_with_nomenclature()
#         # final_output_dict = {}
#         # final_output_of_cele_rec = []analyzer = VideoDetect(roleArn, bucket, "", 0)
#     # Create SNS topic and SQS queue once
        
#         analyzer = VideoDetect(roleArn, bucket, "", 0)
#         analyzer.CreateTopicandQueue()

    
#         final_output_of_content_mode = []

#         for video_segment in list_video_segments:
#             video = video_segment.split('/')[-1]
#             segment = extract_segment_number(video)
#         # Update the video and segment for the analyzer
#             analyzer.video = video
#             analyzer.segment = segment
#             print('Content Moderation Summary:')
#         # The following line was commented out in your code, but I assume you want to use it
#         # analyzer.StartCelebrityRecognition()
#             if analyzer.GetSQSMessageSuccess()==True:
#                 output_of_Celebrity_rec = analyzer.GetCelebrityRekognitionResults() 
#                 final_output_of_content_mode.append(output_of_Celebrity_rec)
#     # Delete SNS topic and SQS queue after processing all segments
#         analyzer.DeleteTopicandQueue()
#         return final_output_of_content_mode





In [16]:
def Celebrity_reko(list_video_segments):
    # try:
        print("list_video_segments "*80)
        print(list_video_segments)
        save_path = 'D:/Demo_Resources/Moderation_AI/video/'
        # json_path = 'D:/Demo_Resources/Transcription_AI/Transcript_flask/json_file/'

        # split_video_into_segments(path_of_video)
        # list_video_segments = get_videos_with_nomenclature()
        final_output_dict = {}
        final_output_of_cele_rec = []
        final_output_of_content_mode = []

        for video_segment in list_video_segments:
            # upload_to_s3(path_of_video)

            roleArn = 'arn:aws:iam::<aws_account_id>:role/AmazonRekognitionVideoServiceRole'
            bucket = '<aws_bucket_name>'
            # UNCOMMENT THIS FOR SHORT VIDEO
            # video = video_file.filename
            video = video_segment
            video = video.split('/')[-1]
            segment = extract_segment_number(video)

            print('Content Moderation Summary:')

            analyzer = VideoDetect(roleArn, bucket, video,segment)
            analyzer.CreateTopicandQueue()

            output_value_of_srt = None

            analyzer.StartCelebrityRecognition()
            for i in range(1):
                if analyzer.GetSQSMessageSuccess()==True:
                    output_of_Celebrity_rec=analyzer.GetCelebrityRekognitionResults()
                    # output_of_Content_moderation = analyzer.GetContentModerationResults()
                    print('Right before break!!')
                    # final_output_of_content_mode.append(output_of_Content_moderation)
                    final_output_of_cele_rec.append(output_of_Celebrity_rec)
                    break
            analyzer.DeleteTopicandQueue()
            print(final_output_of_cele_rec)
        # for video_segment in list_video_segments:
        #     delete_temp_file(video_segment)
        # final_output_dict["cele_output"] = final_output_of_cele_rec
        # final_output_dict["Content_mode"] = final_output_of_content_mode
        
        return final_output_of_cele_rec
    # except Exception as e:
    #     # Log the exception for debugging purposes
    #     print(f"Exception occurred: {e}")
    #     return str(e), 500


In [17]:
# testing_thread_1(['D:/Demo_Resources/Moderation_AI/video/Patal_lok_segment_1.mp4', 'D:/Demo_Resources/Moderation_AI/video/Patal_lok_segment_2.mp4', 'D:/Demo_Resources/Moderation_AI/video/Patal_lok_segment_3.mp4', 'D:/Demo_Resources/Moderation_AI/video/Patal_lok_segment_4.mp4', 'D:/Demo_Resources/Moderation_AI/video/Patal_lok_segment_5.mp4']
# )

In [18]:
def content_mode(list_video_segments):
    try:
        print("list_video_segments "*80)
        print(list_video_segments)
        save_path = 'D:/Demo_Resources/Moderation_AI/video/'
        # json_path = 'D:/Demo_Resources/Transcription_AI/Transcript_flask/json_file/'
        # split_video_into_segments(path_of_video)
        # list_video_segments = get_videos_with_nomenclature()
        # final_output_dict = {}
        # final_output_of_cele_rec = []
        final_output_of_content_mode = []

        for video_segment in list_video_segments:
            # upload_to_s3(path_of_video)

            roleArn = 'arn:aws:iam::<aws_account_id>:role/AmazonRekognitionVideoServiceRole'
            bucket = '<aws_bucket_name>'
            # UNCOMMENT THIS FOR SHORT VIDEO
            # video = video_file.filename
            video = video_segment
            video = video.split('/')[-1]
            segment = extract_segment_number(video)


            print('Content Moderation Summary:')

            analyzer = VideoDetect(roleArn, bucket, video,segment)
            analyzer.CreateTopicandQueue()

            output_value_of_srt = None
            analyzer.StartContentModeration()
            for i in range(1):
                if analyzer.GetSQSMessageSuccess()==True:
                    # output_of_Celebrity_rec=analyzer.GetCelebrityRekognitionResults()
                    output_of_Content_moderation = analyzer.GetContentModerationResults()
                    print('Right before break!!')
                    final_output_of_content_mode.append(output_of_Content_moderation)
                    # final_output_of_cele_rec.append(output_of_Celebrity_rec)
                    break
            analyzer.DeleteTopicandQueue()
            print("final_output_of_content_mode "*80)
            print(final_output_of_content_mode)
        # for video_segment in list_video_segments:
        #     delete_temp_file(video_segment)
        # final_output_dict["cele_output"] = final_output_of_cele_rec
        # final_output_dict["Content_mode"] = final_output_of_content_mode
        
        return final_output_of_content_mode
    except Exception as e:
        # Log the exception for debugging purposes
        print(f"Exception occurred: {e}")
        return str(e), 500



In [19]:
def srt_generation(video_file):
    # try:
        video_file = video_file
        
        save_path = 'D:/Demo_Resources/Moderation_AI/video/'
        json_path = 'D:/Demo_Resources/Moderation_AI/json_file/'
        
        path_of_video = (save_path + video_file.filename)
        key = "azure_id_key"
        region = "centralindia"
        auto_detect_source_language_config = speechsdk.languageconfig.AutoDetectSourceLanguageConfig(
        languages=["en-IN", "hi-IN",])
        path_of_wav_file = convert_video_to_wav(path_of_video)
        path_of_json = (json_path + video_file.filename)
        path_of_json = path_of_json.replace(".mp4",".json")
        print("path_of_json "*80)
        print(path_of_json)
        transcribe(key,region,auto_detect_source_language_config,path_of_wav_file,path_of_json)
        
        
        list_of_words = convert_json_into_words_by_words_list(path_of_json)
        print("list_of_words "*80)
        print(list_of_words)
        path_of_srt = path_of_video.replace("video","srt_file").replace(".mp4",".srt")
        print("path_of_srt "*80)
        print(path_of_srt)
        output_value_of_srt = generate_srt_file(list_of_words,path_of_srt)

        print(path_of_json)
        print("path_of_wav_file:",path_of_wav_file)
        return output_value_of_srt, 200
    # except Exception as e:
    #     # Log the exception for debugging purposes
    #     print(f"Exception occurred: {e}")
    #     return str(e), 500


In [20]:
def Monuments_detection(video_path, model_path = "./monument_1.pt"):
    classes = ['1000 Pillar Temple Moodabidri', 'Aaliyar Dam Park', 'Academy of Fine Arts Kolkata', 'Achalgarh Fort', 'Achut Bibis Mosque', 'Adalaj Stepwell', 'Adam Fountain', 'Adhanur temple', 'Adhi Ratneswarar Temple', 'Adina Mosque', 'Adinath Temple', 'Adiyogi Shiva statue', 'Afghan Church', 'Aga Khan Palace', 'Agra Fort', 'Agrasen Ki Baoli', 'Ahinsa Sthal', 'Aina Mahal', 'Airavateswarar Temple', 'Ajanta Caves', 'Akal Takht Amritsar', 'Akbar Tomb', 'Akshardham temple', 'Alamgir Masjid', 'Albert Hall Museum', 'Alipore Zoological Gardens', 'All Saints, Cathedral', 'Allahabad Fort', 'Allahabad Public Library', 'Alwar Qila', 'Amarnath Temple', 'Ambalappuzha Sri Krishna Temple', 'Ambedkar Park', 'Amber fort', 'Amer Fort', 'Amruteshwar Temple', 'Ananta Basudeba Temple', 'Ananta Vasudev Temple', 'Ananthapura Lake Temple', 'Anna Memorial', 'Antillia', 'Armenian Church', 'Arthanareeswarar Temple', 'Arulmigu Arunachaleswarar Temple', 'Ashtalakshmi Temple', 'Asirgarh Fort', 'Athirappilly Falls', 'Bada Bagh', 'Bada Gumbad', 'Bada Imambara', 'Badami Cave Temples', 'Badrinath Temple', 'Balaji Temple Ketkawale', 'Bandel Church', 'Bandra Fort', 'Bandra Worli Sea Link', 'Baneshwar Mandir', 'Bangalore Fort', 'Bangalore Palace', 'Bangalore Town Hall', 'Bankey Bihari Temple', 'Bara Kaman', 'Baramotichi Vihir', 'Baroda Art Museum', 'Baroli Temple Complex', 'Basilica of Bom Jesus', 'Basilica of Our Lady of Dolours', 'Basilica of Our Lady of Graces', 'Basilica of Our Lady of Ransom, Vallarpadam', 'Basilica of Our Lady of Snows, Pallippuram', 'Basilica of Our Lady of the Assumption, Secunderabad', 'Bateshwar Group of Temples', 'Begumpuri Masjid', 'Bekal Fort', 'Belur Math', 'Bhadra Fort', 'Bhairavakona', 'Bhakti Dham Mangarh', 'Bhandasar Jain Temple', 'Bharathi Park', 'Bhavanisagar Dam', 'Bhimas-Ratha', 'Bhimakali Temple', 'Bhimashankar', 'Bhimeshvara Temple Nilagunda', 'Bhoganandishwara Temple', 'Bhojeshwar Temple', 'Bhor Rajwada', 'Bhoramdev Temple', 'Bhuleshwar Temple', 'Bhutanatha group of temples', 'Bidar Fort', 'Biodiversity Park', 'Bir Singh Deo Palace', 'Bir-kangra', 'Birla House Gandhi Smriti', 'Birla Mandir, Hyderabad', 'Birla Mandir, Jaipur', 'Birla Mandir, Kolkata', 'Birla Planetarium', 'Bisaldeo Temple', 'Bnaganga Tank', 'Bogatha Waterfalls', 'Bojjannakonda', 'Bolgatty Palace', 'Bombay Stock Exchane', 'Brahma Temple, Pushkar', 'Brahmesvara Temple', 'Brahmeswara Temple', 'Brihadeeswara Temple', 'Buddha International Circuit', 'Buddha Park of Ravangla', 'Buddha Smriti Park', 'Bull Temple', 'Central Bureau of Investigation', 'Char Minar', 'Chhatrapati Shivaji Terminus', 'City Palace', 'Dagdusheth Halwai Ganpati Temple', 'Dakshineshwar Kali Temple', 'Dakshineswar Kali Temple', 'Dansborg Fort', 'Dargah Aala Hazrat', 'Daria Daulat Bagh', 'Das Mahavidya temple', 'Dashashwamedh Ghat', 'Dassam Falls', 'Dastgeer Sahib', 'Daulatabad, Maharashtra', 'David Sassoon Library', 'Deekshabhoomi', 'Delhi Gate', 'Delhi Town Hall', 'Descent of the Ganges', 'Devanahalli Fort', 'Devi Jagdambi Temple Khajuraho', 'Dhamekh Stupa', 'Dharmaraja Ratha', 'Dharmaraya Swamy Temple', 'Dhom Dam', 'Dhrasanvel Temple, Magderu', 'Dhuandhar Falls', 'Diu Fort', 'Do Drul Chorten', 'Dr MGR Memorial', 'Dras War', 'Dubdi Gompa', 'Dudhsagar Falls', 'DulhaDeo Temple Khajuraho', 'Durga Mandir in Bhelupur', 'Durga Temple, Aihole', 'Dwarkadhish Temple', 'Elephanta Caves', 'Ellora caves', 'Enchey Monastery', 'Eros Cinema', 'Erumbeeswarar Temple', 'Ethipothala Falls', 'Ettumanoor Temple', 'Fatehpur sikri', 'Feroz shah kotla', 'Fort Unchagaon', 'Gateway of India', 'Gobal Vipassana Pagoda', 'Gol Gumbaz', 'Golconda fort', 'Golden Temple', 'Gurudwara Bangla Sahib', 'Gurudwara Sahib, Pushkar', 'Haji ali dargha', 'Hawa Mahal', 'High Court of Karnatka', 'Howrah Bridge', 'Humayu Tomb', 'India gate', 'Jal Mahal', 'Jallianwala Bagh', 'Jama Mashjid', 'Jamia Masjid or Begumpet Mosque', 'Jantar Mantar, Jaipur', 'Jantar Mantar', 'Khajuraho Temple', 'Konark Sun Temple', 'Laxmi Narayan Mandir', 'Lotus Temple', 'Mecca Masjid Hyderabad', 'Mehrangarh Fort', 'Mysore Palace', 'Parliament of India', 'Qutub Minar', 'Rashtrapati Bhavan', 'Red Fort', 'Sajjangarh Monsoon Palace', 'Samudra Narayan Temple', 'Sanchi Stupa', 'Shiv Niwas Palace', 'Shore Temple', 'Shree Banashankari Amma Temple', 'Sidhi Vinayak', 'Supreme Court', 'Taj Mahal', 'The Imperial Tower', 'The Taj Mahal palace hotels', 'Umaid Bhawan Palace', 'Vasai Fort Or Bassien Fort', 'Victoria Memorial', 'Vidhana Soudha']

# def detect_objects_in_video(video_path, model_path, classes):
    # Load the model
    # model = torch.hub.load('ultralytics/yolov5', 'custom', path_or_model=model_path)
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)
    model.eval()

    # Open the video
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    results = {}

    for i in range(0, total_frames, fps):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()

        if not ret:
            break

        # Detect objects in the frame
        detections = model(frame)
        # results = detections

        # Convert detections to desired format
        detected_objects = []

        # the part we got from gpt-4:
        for *xyxy, conf, cls in detections.xyxy[0].cpu().numpy():
            detected_objects.append({
                'coordinates': xyxy,
                'confidence': conf,
                'class': model.names[int(cls)]
            })

        # the part we implemented in pmayg-simplified code:
        # for nested_result_list in detections.xywhn:
        #     class_dict = {}
        #     for result_tensor in nested_result_list:
        #         # class_index = int(result_tensor[-1])
        #         # class_probability = float(result_tensor[-2])
        #         # if classes[class_index] in class_dict:
        #         #     if class_dict[classes[class_index]] < class_probability:
        #         #         class_dict[classes[class_index]] = class_probability
        #         #     else:
        #         #         class_dict[classes[class_index]] = class_probability
        #         # else:
        #         #     class_dict[classes[class_index]] = class_probability
        #         # detected_objects.append(class_dict)
        #         detected_objects.append(result_tensor)


        # Add the results to the dictionary
        if len(detected_objects)>0:
            timestamp = i // fps
            results["timestamp"] = detected_objects
        print("detected_objects:",len(detected_objects))

    cap.release()
    if len(results)==0:
        return "Empty"
    else:
        print("type_in_monument_function:",type(results))
        return results

    
    # classes_monuments = ['1000 Pillar Temple Moodabidri', 'Aaliyar Dam Park', 'Academy of Fine Arts Kolkata', 'Achalgarh Fort', 'Achut Bibis Mosque', 'Adalaj Stepwell', 'Adam Fountain', 'Adhanur temple', 'Adhi Ratneswarar Temple', 'Adina Mosque', 'Adinath Temple', 'Adiyogi Shiva statue', 'Afghan Church', 'Aga Khan Palace', 'Agra Fort', 'Agrasen Ki Baoli', 'Ahinsa Sthal', 'Aina Mahal', 'Airavateswarar Temple', 'Ajanta Caves', 'Akal Takht Amritsar', 'Akbar Tomb', 'Akshardham temple', 'Alamgir Masjid', 'Albert Hall Museum', 'Alipore Zoological Gardens', 'All Saints, Cathedral', 'Allahabad Fort', 'Allahabad Public Library', 'Alwar Qila', 'Amarnath Temple', 'Ambalappuzha Sri Krishna Temple', 'Ambedkar Park', 'Amber fort', 'Amer Fort', 'Amruteshwar Temple', 'Ananta Basudeba Temple', 'Ananta Vasudev Temple', 'Ananthapura Lake Temple', 'Anna Memorial', 'Antillia', 'Armenian Church', 'Arthanareeswarar Temple', 'Arulmigu Arunachaleswarar Temple', 'Ashtalakshmi Temple', 'Asirgarh Fort', 'Athirappilly Falls', 'Bada Bagh', 'Bada Gumbad', 'Bada Imambara', 'Badami Cave Temples', 'Badrinath Temple', 'Balaji Temple Ketkawale', 'Bandel Church', 'Bandra Fort', 'Bandra Worli Sea Link', 'Baneshwar Mandir', 'Bangalore Fort', 'Bangalore Palace', 'Bangalore Town Hall', 'Bankey Bihari Temple', 'Bara Kaman', 'Baramotichi Vihir', 'Baroda Art Museum', 'Baroli Temple Complex', 'Basilica of Bom Jesus', 'Basilica of Our Lady of Dolours', 'Basilica of Our Lady of Graces', 'Basilica of Our Lady of Ransom, Vallarpadam', 'Basilica of Our Lady of Snows, Pallippuram', 'Basilica of Our Lady of the Assumption, Secunderabad', 'Bateshwar Group of Temples', 'Begumpuri Masjid', 'Bekal Fort', 'Belur Math', 'Bhadra Fort', 'Bhairavakona', 'Bhakti Dham Mangarh', 'Bhandasar Jain Temple', 'Bharathi Park', 'Bhavanisagar Dam', 'Bhimas-Ratha', 'Bhimakali Temple', 'Bhimashankar', 'Bhimeshvara Temple Nilagunda', 'Bhoganandishwara Temple', 'Bhojeshwar Temple', 'Bhor Rajwada', 'Bhoramdev Temple', 'Bhuleshwar Temple', 'Bhutanatha group of temples', 'Bidar Fort', 'Biodiversity Park', 'Bir Singh Deo Palace', 'Bir-kangra', 'Birla House Gandhi Smriti', 'Birla Mandir, Hyderabad', 'Birla Mandir, Jaipur', 'Birla Mandir, Kolkata', 'Birla Planetarium', 'Bisaldeo Temple', 'Bnaganga Tank', 'Bogatha Waterfalls', 'Bojjannakonda', 'Bolgatty Palace', 'Bombay Stock Exchane', 'Brahma Temple, Pushkar', 'Brahmesvara Temple', 'Brahmeswara Temple', 'Brihadeeswara Temple', 'Buddha International Circuit', 'Buddha Park of Ravangla', 'Buddha Smriti Park', 'Bull Temple', 'Central Bureau of Investigation', 'Char Minar', 'Chhatrapati Shivaji Terminus', 'City Palace', 'Dagdusheth Halwai Ganpati Temple', 'Dakshineshwar Kali Temple', 'Dakshineswar Kali Temple', 'Dansborg Fort', 'Dargah Aala Hazrat', 'Daria Daulat Bagh', 'Das Mahavidya temple', 'Dashashwamedh Ghat', 'Dassam Falls', 'Dastgeer Sahib', 'Daulatabad, Maharashtra', 'David Sassoon Library', 'Deekshabhoomi', 'Delhi Gate', 'Delhi Town Hall', 'Descent of the Ganges', 'Devanahalli Fort', 'Devi Jagdambi Temple Khajuraho', 'Dhamekh Stupa', 'Dharmaraja Ratha', 'Dharmaraya Swamy Temple', 'Dhom Dam', 'Dhrasanvel Temple, Magderu', 'Dhuandhar Falls', 'Diu Fort', 'Do Drul Chorten', 'Dr MGR Memorial', 'Dras War', 'Dubdi Gompa', 'Dudhsagar Falls', 'DulhaDeo Temple Khajuraho', 'Durga Mandir in Bhelupur', 'Durga Temple, Aihole', 'Dwarkadhish Temple', 'Elephanta Caves', 'Ellora caves', 'Enchey Monastery', 'Eros Cinema', 'Erumbeeswarar Temple', 'Ethipothala Falls', 'Ettumanoor Temple', 'Fatehpur sikri', 'Feroz shah kotla', 'Fort Unchagaon', 'Gateway of India', 'Gobal Vipassana Pagoda', 'Gol Gumbaz', 'Golconda fort', 'Golden Temple', 'Gurudwara Bangla Sahib', 'Gurudwara Sahib, Pushkar', 'Haji ali dargha', 'Hawa Mahal', 'High Court of Karnatka', 'Howrah Bridge', 'Humayu Tomb', 'India gate', 'Jal Mahal', 'Jallianwala Bagh', 'Jama Mashjid', 'Jamia Masjid or Begumpet Mosque', 'Jantar Mantar, Jaipur', 'Jantar Mantar', 'Khajuraho Temple', 'Konark Sun Temple', 'Laxmi Narayan Mandir', 'Lotus Temple', 'Mecca Masjid Hyderabad', 'Mehrangarh Fort', 'Mysore Palace', 'Parliament of India', 'Qutub Minar', 'Rashtrapati Bhavan', 'Red Fort', 'Sajjangarh Monsoon Palace', 'Samudra Narayan Temple', 'Sanchi Stupa', 'Shiv Niwas Palace', 'Shore Temple', 'Shree Banashankari Amma Temple', 'Sidhi Vinayak', 'Supreme Court', 'Taj Mahal', 'The Imperial Tower', 'The Taj Mahal palace hotels', 'Umaid Bhawan Palace', 'Vasai Fort Or Bassien Fort', 'Victoria Memorial', 'Vidhana Soudha']
    # classes_logo = ['ACKO', 'Adidas', 'Amazon', 'Amul', 'Aramco', 'BCCI', 'Booking.com', 'Byjus', 'CRED', 'Ceat', 'Chennai Super kings', 'Deccan Chargers', 'Delhi Capitals', 'Domino-s','Dream11', 'EBIX Cash', 'Emirates', 'Fbb','Go Daddy','Gujarat Titans','Gujarat lions','Gulf','Hotstar','IPL','JK lakshmi cement','Jio','Kochi Tuskers','Kolkata Knigth Riders','Lucknow Super Giants','MPL','MRF','Mountain Dew','Mumbai Indians','Myntra','Oppo','Paytm','Pepsi','Postpe','Pune Warriors','Punjab Kings','Rajasthan Royals','Rising Pune Supergiants','Royal Challengers Bangalore','Rupay','Slice IPL','Star Sports','Sunrisers Hyderabad','Swiggy', 'TATA Safari', 'TATA', 'TVS Eurogrip' , 'Unacademy', 'Unicef'    , 'Upstox', 'Vivo', 'safari']
    # model_monuments = torch.hub.load('ultralytics/yolov5', 'custom', path='./monument_1.pt')
    # model_label = torch.hub.load('ultralytics/yolov5', 'custom', path='./logo.pt')
    # video_path = video_path
    # # print("video_path "*80)
    # # print(video_path)
    # cap = cv2.VideoCapture(video_path)

    # frame_rate = cap.get(cv2.CAP_PROP_FPS)
    # print("frame_rate:",frame_rate)
    # frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # monument_results = []
    # logo_results = []
    # dict_for_monum = {}
    # dict_for_logo = {}
    # for second in range(int(frame_count / frame_rate)):
        
    #     print("second:",second)

    #     start_frame = int(second * frame_rate)
    #     end_frame = int((second + 1) * frame_rate) - 1

    #     total_confidence = 0
    #     monument_names = set()
    #     logo_names = set()

    #     for frame_idx in range(start_frame, end_frame + 1):
    #         ret, frame = cap.read()
    #         if not ret:
    #             break

    #         # Convert the frame to PIL Image
    #         image = Image.fromarray(frame)
    #         image.save("./gdgsdfsf.jpg")

    #         # Perform detection on the frame
    #         results = model_monuments(image)
    #         results2 = model_label(image)
    #         for nested_result_list in results.xywhn:
    #             for result_tensor in nested_result_list:
    #                 class_index = int(result_tensor[-1])
    #                 class_probability = float(result_tensor[-2])
    #                 class_name = classes_monuments[class_index]
    #                 total_confidence += class_probability
    #                 monument_names.add(class_name)
    #                 print("monuments_name:",monument_names)
        
    #         for nested_result_list in results2.xywhn:
    #             for result_tensor in nested_result_list:
    #                 class_index = int(result_tensor[-1])
    #                 class_probability = float(result_tensor[-2])
    #                 class_name = classes_logo[class_index]
                    
    #                 total_confidence += class_probability
    #                 logo_names.add(class_name)
    #                 print("logo_name:",logo_names)
    #     dict_for_monum[second] = monument_names
    #     dict_for_logo[second] = logo_names            

    #     avg_confidence = total_confidence / frame_rate
    #     timestamp = time.strftime("%H:%M:%S", time.gmtime(second))
    #     monument_names_str = ', '.join(monument_names)

    #     avg_confidence = total_confidence / frame_rate
    #     timestamp = time.strftime("%H:%M:%S", time.gmtime(second))
    #     logo_names_str = ', '.join(logo_names)
    #     if monument_names:
    #         monument_results.append({
    #             'timestamp': timestamp,
    #             'monument_names': monument_names_str,
    #             'confidence': avg_confidence
    #         })
    #     if logo_names:
    #         logo_results.append({
    #             'timestamp': timestamp,
    #             'monument_names': logo_names_str,
    #             'confidence': avg_confidence
    #         })
    # cap.release()

    # return monument_results,logo_results
    


In [21]:
# ky = (testing_thread_4("D:/Chrome_downloads/patal_lok_short.mp4"))

In [22]:
# (ky)

In [23]:
def logo_detection(video_path, model_path = "./logo.pt"):
    classes = ['ACKO', 'Adidas', 'Amazon', 'Amul', 'Aramco', 'BCCI', 'Booking.com', 'Byjus', 'CRED', 'Ceat', 'Chennai Super kings', 'Deccan Chargers', 'Delhi Capitals', 'Domino-s','Dream11', 'EBIX Cash', 'Emirates', 'Fbb','Go Daddy','Gujarat Titans','Gujarat lions','Gulf','Hotstar','IPL','JK lakshmi cement','Jio','Kochi Tuskers','Kolkata Knigth Riders','Lucknow Super Giants','MPL','MRF','Mountain Dew','Mumbai Indians','Myntra','Oppo','Paytm','Pepsi','Postpe','Pune Warriors','Punjab Kings','Rajasthan Royals','Rising Pune Supergiants','Royal Challengers Bangalore','Rupay','Slice IPL','Star Sports','Sunrisers Hyderabad','Swiggy', 'TATA Safari', 'TATA', 'TVS Eurogrip' , 'Unacademy', 'Unicef'    , 'Upstox', 'Vivo', 'safari']

# def detect_objects_in_video(video_path, model_path, classes):
    # Load the model
    # model = torch.hub.load('ultralytics/yolov5', 'custom', path_or_model=model_path)
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)
    model.eval()

    # Open the video
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    results = {}

    for i in range(0, total_frames, fps):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()

        if not ret:
            break

        # Detect objects in the frame
        detections = model(frame)
        # results = detections

        # Convert detections to desired format
        detected_objects = []

        # the part we got from gpt-4:
        for *xyxy, conf, cls in detections.xyxy[0].cpu().numpy():
            detected_objects.append({
                'coordinates': xyxy,
                'confidence': conf,
                'class': model.names[int(cls)]
            })
    
        # Add the results to the dictionary
        if len(detected_objects)>0:
            timestamp = i // fps
            results[timestamp] = detected_objects
        print("detected_objects:",len(detected_objects))

    cap.release()
    if len(results)==0:
        return "Empty"
    else:
        print(type(results))
        return results


In [24]:
import cv2
import torch
from google.oauth2 import service_account
import json
from google.cloud import vision
from google.cloud import videointelligence
service_account_info_string = R"""Google account details

"""

service_account_info_json = json.loads(service_account_info_string)
service_account_creds = service_account.Credentials.from_service_account_info(service_account_info_json)

video_client = videointelligence.VideoIntelligenceServiceClient(credentials=service_account_creds)
features = [videointelligence.Feature.SHOT_CHANGE_DETECTION]
def detect_web(path, timestamp,frame_timestamp):
    """Detects web annotations given an image."""
    # from google.cloud import vision
    dict_1 ={}
    client = vision.ImageAnnotatorClient(credentials=service_account_creds)

    # with open(path, "rb") as image_file:
    #     content = image_file.read()

    content = path
    # image = vision.Image(content=content)
    image = vision.Image(content=content)

    response = client.web_detection(image=image)
    annotations = response.web_detection

    if annotations.best_guess_labels:
        for label in annotations.best_guess_labels:
            print(f"\n: {label.label}")
            dict_1["Best_guess_level"] =str(label.label)


    if annotations.web_entities:
        print("\n{} custom_label: ".format(len(annotations.web_entities)))

        for entity in annotations.web_entities:
            entity_1={}

            # print(f"\n\tConfidence    : {entity.score}")
            # print(f"\tDetails: {entity.description}")
            entity_1["Confidence"] = entity.score
            entity_1["Details"] = entity.description
            list_entity = [entity_1]
            print("list_entity:",list_entity)
            if "Custom_label" not in dict_1.keys():
                dict_1["Custom_label"] = []
            dict_1["Custom_label"].append(list_entity[0])
    dict_2 = {}
    dict_2[(frame_timestamp)] =dict_1         
   
    if response.error.message:
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
        )
    return dict_2

def shot_detection(video_path):
    result_list_of_web = []
    with open(video_path, "rb") as f:
        video_data = f.read()
    operation = video_client.annotate_video(
        request={"features": features,  "input_content": video_data}
    )
    print("\nProcessing video for shot change annotations:")

    result = operation.result(timeout=90000)
    print("\nFinished processing.")

    # first result is retrieved because a single video was processed
    
    for i, shot in enumerate(result.annotation_results[0].shot_annotations):
        start_time = (
            shot.start_time_offset.seconds + shot.start_time_offset.microseconds / 1e6
        )
        end_time = (
            shot.end_time_offset.seconds + shot.end_time_offset.microseconds / 1e6
        )
        print("\tShot {}: {} to {}".format(i, start_time, end_time))


    # Extract and save frames at shot change points
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        # fps = data.get(cv2.CAP_PROP_FPS)
  
    print("fps:",fps)
    print("frames:",frames)
    frame_number = 0
    for i, shot in enumerate(result.annotation_results[0].shot_annotations,fps):
        start_time = (
            shot.start_time_offset.seconds + shot.start_time_offset.microseconds / 1e6
        )
        end_time = (
            shot.end_time_offset.seconds + shot.end_time_offset.microseconds / 1e6
        )
        
        # Convert time to frame number
        start_frame = int(start_time * cap.get(cv2.CAP_PROP_FPS))
        end_frame = int(end_time * cap.get(cv2.CAP_PROP_FPS))
        
        # Save the frame at the shot change point
        middle_frame = int((start_frame + end_frame) / 2)  # Save the middle frame
        print("frame_num:",middle_frame)
        cap.set(cv2.CAP_PROP_POS_FRAMES, middle_frame)
        frame_timestamp = start_time + (middle_frame / cap.get(cv2.CAP_PROP_FPS))
        ret, frame = cap.read()
        print("frame:",frame)
        print("frame_timestamp:",frame_timestamp)
        ret, buffer = cv2.imencode('.jpg', frame)
        image_data = buffer.tobytes()
  
        print("i//fps:",i//fps)
        # Pass the image data to Google Vision API
        result_list_of_web.append(detect_web(image_data, i // fps,frame_timestamp))

        
        if ret:
            frame_filename = f"./shot_img/shot_change_frame_{frame_number:04d}.jpg"
            cv2.imwrite(frame_filename, frame)
            print(f"Saved {frame_filename}")
            frame_number += 1

    cap.release()
    return result_list_of_web

In [25]:
# rest = detect_objects_in_video("D:/Chrome_downloads/output_video_gow.mp4")

In [26]:
# monuments,logo=testing_thread_4("D:/Chrome_downloads/video6224143550255204691.mp4")

In [27]:
import numpy as np

# Recursive function to convert float32 values to float
def convert_float32_to_float(item):
    if isinstance(item, dict):
        return {key: convert_float32_to_float(value) for key, value in item.items()}
    elif isinstance(item, list):
        return [convert_float32_to_float(value) for value in item]
    elif isinstance(item, np.float32):
        return float(item)
    else:
        return item


In [28]:
import json
from itertools import groupby


def process_json(json_content):
    # with open(input_file, "r") as file:
    #     json_content = json.load(file)
    # print("+"*80)
    # print(json_content)
    # Flatten the list of lists
    flattened_data = [item for sublist in json_content for item in sublist]

    # Sort the flattened list by 'Moderated Label Name' and 'timestamp'
    sorted_data = sorted(flattened_data, key=lambda x: (x['Moderated Label Name'], x['timestamp']))

    # Helper function to convert milliseconds to HH:MM:SS format
    def ms_to_hms(ms):
        seconds, ms = divmod(ms, 1000)
        minutes, seconds = divmod(seconds, 60)
        hours, minutes = divmod(minutes, 60)
        return f"{hours:02}:{minutes:02}:{seconds:02}"

    # Group by 'Moderated Label Name'
    grouped_data = {}
    for key, group in groupby(sorted_data, key=lambda x: x['Moderated Label Name']):
        intervals = []
        start_time = None
        prev_time = None
        for item in group:
            if start_time is None:
                start_time = item['timestamp']
            elif item['timestamp'] - prev_time > 1000:  # If there's a gap larger than 1 second
                intervals.append((start_time, prev_time))
                start_time = item['timestamp']
            prev_time = item['timestamp']
        intervals.append((start_time, prev_time))
        grouped_data[key] = intervals

    # Convert intervals to desired format
    output_data = []
    for label, intervals in grouped_data.items():
        time_ranges = ", ".join([f"{ms_to_hms(start)} - {ms_to_hms(end)}" for start, end in intervals])
        output_data.append({label: time_ranges})

    # # Save to output file
    # with open(output_file, "w") as outfile:
    #     json.dump(output_data, outfile, indent=4)
    return output_data
# Example usage:
# process_json("input.json", "output.json")
# process_json("output_video_gow.json", "segmented_output_video_gow.json")
# process_json("6 minutes of brutal movie gore (5).json", "segmented_6 minutes of brutal movie gore (5).json")
# process_json("Patal_lok_vid.json", "segmented_Patal_lok_vid.json")
# process_json("THOMAS_SHELBY_SMOKING.json", "segmented_THOMAS_SHELBY_SMOKING.json")
# process_json("output_video_gow.json", "segmented_videoplayback (1).json")
 

In [29]:
from flask import Flask, request, jsonify
app = Flask(__name__)

@app.route('/upload_video', methods=['GET', 'POST'])
def upload_video():
    # try:

        content_headers = request.headers.get("Content-Moderation")
        cele_rek_headers = request.headers.get("Celebrity-Rek")
        srt_headers = request.headers.get("Srt-Generation")
        monument_headers = request.headers.get("Monument-detection")
        logo_headers = request.headers.get("Logo-detection")
        custom_headers = request.headers.get('My-Custom-Header')
        print("headers:", request.headers)
        print("custom_headers:",custom_headers)

        if 'video' not in request.files:
            return 'No video file found', 400
        save_path = 'D:/Demo_Resources/Moderation_AI/video/'
        
        # destination_directory = 'D:/Demo_Resources/Moderation_AI/video/'
        
        # Save the uploaded video to the new destination directory
        # video_file = request.files['video']
        # video_file.save(os.path.join(destination_directory, video_file.filename))
        

        video_file = request.files['video']

        # Save the video to the specified path
        video_file.save(save_path + video_file.filename)
        path_of_video = (save_path + video_file.filename)
        print("start split_video_into_segments ")
        split_video_into_segments(path_of_video)
        
        print("complete split_video_into_segments ")
        
        print("start get_videos_with_nomenclature ")
        list_video_segments = get_videos_with_nomenclature()
        print("Complete ")

        # path_of_wav_file = convert_video_to_wav(path_of_video)
# =====================================================================================
        # threads = []
        # results_function1 = []
        # results_function2 = []
        # results_function3 = []                
        # results_function4 = []
        # print("results_function4 :",type(results_function4))
        # results_function5 = []
        # function1_args = [list_video_segments,]
        # function2_args =  [list_video_segments,]
        # function3_args = [video_file,]
        # function4_args = [path_of_video,]
        # function5_args = [path_of_video,]
# ===================================================
        # presenttime = timex.time()
        # presenttime = time.time()  
        result_dict ={}
        
        # results_function2 =content_mode(list_video_segments)
        # print("done:","execute_function_of_content_mode")
        # results_function1 = Celebrity_reko(list_video_segments)
        # print("done:","execute_function_of_Celebrity_reko")
        # results_function3 =srt_generation(video_file)
        # print("done:","execute_function_of_srt_generation")
        # results_function4 =Monuments_detection(path_of_video)
        # print("done:","execute_function_for_monument")
        # results_function5 =logo_detection(path_of_video)
        # print("done:","execute_function_for_logo")
        # results_function6 =shot_detection(path_of_video)
        # print("done:","execute_function_for_logo")

        if content_headers == "enable": 
            results_function2 = process_json(content_mode(list_video_segments))
            print("done","execute_function_of_content_mode")
            
            result_dict["content_mode"] = (results_function2)
        if content_headers == "disable":
            result_dict["content_mode"] = "disable"
        if cele_rek_headers == "enable":
            results_function1 = Celebrity_reko(list_video_segments)
            print("done","execute_function_of_Celebrity_reko")
            result_dict["celebrity_reko"] = (results_function1)
        if cele_rek_headers == "disable":
            
            result_dict["celebrity_reko"] = "disable"
        if srt_headers == "enable":
            results_function3 =srt_generation(video_file)
            print("done","execute_function_of_srt_generation")
            result_dict["srt"] = (results_function3)
        if srt_headers == "disable":
            result_dict["srt"] = "disable"
        if monument_headers == "enable":
            results_function4=Monuments_detection(path_of_video)
            print("done","execute_function_for_monument")
            converted_dict4 = convert_float32_to_float(results_function4)
            result_dict["monument_result"] = (converted_dict4)

        if monument_headers == "disable":
            result_dict["monument_result"] = "disable"
        if logo_headers == "enable":
            results_function5 =logo_detection(path_of_video)
            print("done","execute_function_for_logo")
            converted_dict = convert_float32_to_float(results_function5)
            result_dict["logo_result"] =   (converted_dict)
        if logo_headers == "disable":
            result_dict["logo_result"] =  "disable"
        if custom_headers == "enable":
             
            results_function6 =shot_detection(path_of_video)
            converted_dict = convert_float32_to_float(results_function6)
            result_dict["web_entities"] =  converted_dict
            print("done:","execute_function_for_logo")
             
        if custom_headers == "disable":
            result_dict["web_entities"] =  "disable"
        # presenttime = time.time()
# ===================================================
                                    
        
            # for arg in function1_args:
            #     thread = threading.Thread(target=lambda arg=arg: results_function1.append(execute_function_of_Celebrity_reko(arg)))
            #     threads.append(thread)
            #     thread.start()    
        # for arg in function2_args:
        #     thread = threading.Thread(target=lambda arg=arg: results_function2.append(execute_function_of_content_mode(arg)))
        #     threads.append(thread)
        #     thread.start()
        # for arg in function3_args:
        #     thread = threading.Thread(target=lambda arg=arg: results_function3.append(execute_function_of_srt_generation(arg)))
        #     threads.append(thread)
        #     thread.start()
        # for arg in function4_args:
        #     thread = threading.Thread(target=lambda arg=arg: results_function4.append(execute_function_for_monument(arg)))
        #     threads.append(thread)
        #     thread.start()  
        # for arg in function5_args:
        #     thread = threading.Thread(target=lambda arg=arg: results_function5.append(execute_function_for_logo(arg)))
        #     threads.append(thread)
        #     thread.start()


        
                
        # for thread in threads:
        #     thread.join()
        # print("results_function1"*80)
        # print("Results:", results_function1)
        # result_dict ={}
        # result_dict["celebrity_reko"] = (results_function1)
        # result_dict["content_mode"] = (results_function2)
        # result_dict["srt"] = (results_function3)
        # # print(results_function4)
        # # print("after_results_dict",type(results_function4))
        # converted_dict4 = convert_float32_to_float(results_function4)
        # result_dict["monument_result"] = (converted_dict4)
        # # print("results_function5:",     (results_function5))
        # # results_function5_ser = {key: float(value) for key, value in results_function5.items()}
        # converted_dict = convert_float32_to_float(results_function5)
        # result_dict["logo_result"] =   (converted_dict)
        # converted_dict = convert_float32_to_float(results_function6)
        # result_dict["web_entities"] =   (converted_dict)
        # print("results_function2"*80)
        # print("Results:", results_function2)
        # print(result_dict)

        return result_dict

    # except Exception as e:
    #     # Log the exception for debugging purposes
    #     app.logger.error(f"Exception occurred: {e}")
    #     return str(e), 500

if __name__ == '__main__':
    app.run()
    # print("="*100)
    # print(a)\
 

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


headers: My-Custom-Header: disable
Content-Moderation: enable
Logo-Detection: disable
Monument-Detection: disable
Srt-Generation: disable
Celebrity-Rek: enable
User-Agent: PostmanRuntime/7.32.3
Accept: */*
Postman-Token: f25ab21e-286d-4253-8cdb-213f1bc277a1
Host: 127.0.0.1:5000
Accept-Encoding: gzip, deflate, br
Connection: keep-alive
Content-Type: multipart/form-data; boundary=--------------------------877650444939204025748188
Content-Length: 37040375


custom_headers: disable
start split_video_into_segments 
Video splitting completed successfully!
complete split_video_into_segments 
start get_videos_with_nomenclature 
Complete 
list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments list_video_segments li

127.0.0.1 - - [11/Aug/2023 10:40:46] "POST /upload_video HTTP/1.1" 200 -


[[{'timestamp_in_proper_format': '-1:59:00', 'timestamp': -59920, 'Celeb Name': 'Manoj Bajpai', 'Confidence': 99.17089080810547}, {'timestamp_in_proper_format': '-1:59:00', 'timestamp': -59440, 'Celeb Name': 'Manoj Bajpai', 'Confidence': 98.02648162841797}, {'timestamp_in_proper_format': '-1:59:01', 'timestamp': -58920, 'Celeb Name': 'Manoj Bajpai', 'Confidence': 98.68854522705078}, {'timestamp_in_proper_format': '-1:59:01', 'timestamp': -58440, 'Celeb Name': 'Manoj Bajpai', 'Confidence': 95.54681396484375}, {'timestamp_in_proper_format': '-1:59:02', 'timestamp': -57920, 'Celeb Name': 'Manoj Bajpai', 'Confidence': 95.84172058105469}, {'timestamp_in_proper_format': '-1:59:03', 'timestamp': -56920, 'Celeb Name': 'Ilya Prusikin', 'Confidence': 88.02826690673828}, {'timestamp_in_proper_format': '-1:59:03', 'timestamp': -56440, 'Celeb Name': 'Manoj Bajpai', 'Confidence': 92.60256958007812}, {'timestamp_in_proper_format': '-1:59:04', 'timestamp': -55440, 'Celeb Name': 'Manoj Bajpai', 'Confid

In [ ]:
# import concurrent.futures
# app = Flask(__name__)

# @app.route('/upload_video', methods=['POST'])
# def upload_video():
#     # try:
#         if 'video' not in request.files:
#             return 'No video file found', 400
#         save_path = 'D:/Demo_Resources/Moderation_AI/video/'
        
#         # destination_directory = 'D:/Demo_Resources/Moderation_AI/video/'
        
#         # Save the uploaded video to the new destination directory
#         # video_file = request.files['video']
#         # video_file.save(os.path.join(destination_directory, video_file.filename))
        

#         video_file = request.files['video']

#         # Save the video to the specified path
#         video_file.save(save_path + video_file.filename)
#         path_of_video = (save_path + video_file.filename)
#         split_video_into_segments(path_of_video)
#         list_video_segments = get_videos_with_nomenclature()



#         with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
#             args = [list_video_segments, list_video_segments, video_file, path_of_video, path_of_video]

#             # Submit the functions to the executor and store the Future objects
#             futures = [executor.submit(func, arg) for func, arg in zip([Celebrity_reko, content_mode, srt_generation, Monuments_detection, logo_detection], args)]

#             # Retrieve the results from the Future objects
#             results = [future.result() for future in futures]

# # Print the results
#         for result in results:
#             print(result)

#         return results

# if __name__ == '__main__':
#     app.run()